### `tf.Tensor` and `tf.Variable`

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

In [4]:
a = tf.constant([[1,2,3,4]], dtype=tf.int32)
tf.transpose(a) @ a

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 1,  2,  3,  4],
       [ 2,  4,  6,  8],
       [ 3,  6,  9, 12],
       [ 4,  8, 12, 16]], dtype=int32)>

In [5]:
tf.concat([a,a,a], axis=-1)

<tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]], dtype=int32)>

In [6]:
v = tf.Variable(a)
v.assign_add(a)

<tf.Variable 'UnreadVariable' shape=(1, 4) dtype=int32, numpy=array([[2, 4, 6, 8]], dtype=int32)>

In [7]:
# Shape returns a `TensorShape` object that shows the size along each axis
x = tf.constant([[1], [2], [3]])
print(x.shape)

(3, 1)


In [8]:
@tf.function
def my_func(x):
  print('Tracing.\n')
  return tf.reduce_sum(x)

# everytime shape or dtype of x changes, a new graph is generated
# non-tf ops are only run once, during the first call of the function, and ignored in all subsequent calls,
# because the the first time the function is called, it's compiled in a static graph which is run on each subsequent call of that function. 
# non-tf ops are not a part of this graph.

In [13]:
my_func(tf.constant([1,2,4]))

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [29]:
tf.saved_model.save(my_func, 'myfunc') 
# only works with objects of subclasses of tf.Module, like keras.Model or keras.layers.Layer
# @tf. functions should only contain tf ops, and avoid python ops

In [32]:
ragged_list = [
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]]

tf.ragged.constant(ragged_list)

<tf.RaggedTensor [[0, 1, 2, 3], [4, 5], [6, 7, 8], [9]]>

In [33]:
# If you have three string tensors of different lengths, this is OK.
tensor_of_strings = tf.constant(["Gray wolf",
                                 "Quick brown fox",
                                 "Lazy dog"])
# Note that the shape is (3,). The string length is not included.
print(tensor_of_strings)

tf.Tensor([b'Gray wolf' b'Quick brown fox' b'Lazy dog'], shape=(3,), dtype=string)


In [35]:
tf.strings.split(tensor_of_strings).shape

TensorShape([3, None])

In [43]:
text = tf.constant("1 10 100")
text

<tf.Tensor: shape=(), dtype=string, numpy=b'1 10 100'>

In [44]:
tf.strings.to_number(tf.strings.split(text, " "))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([  1.,  10., 100.], dtype=float32)>

In [47]:
byte_strings = tf.strings.bytes_split(tf.constant("Duck"))
byte_ints = tf.io.decode_raw(tf.constant("Duck"), tf.uint8)
print("Byte strings:", byte_strings)
print("Bytes:", byte_ints)

Byte strings: tf.Tensor([b'D' b'u' b'c' b'k'], shape=(4,), dtype=string)
Bytes: tf.Tensor([ 68 117  99 107], shape=(4,), dtype=uint8)


### AutoDiff

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

In [3]:
tape.gradient(loss, [w, b])

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.28481627,  1.3564167 ],
        [-0.56963253,  2.7128334 ],
        [-0.8544488 ,  4.06925   ]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.28481627,  1.3564167 ], dtype=float32)>]

In [6]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)
grad

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.       , 0.8670832],
        [0.       , 1.7341664],
        [0.       , 2.6012497]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.       , 0.8670832], dtype=float32)>]

In [8]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.37591663>

In [4]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


In [8]:
[var.name for var in tape.watched_variables()]

['x0:0']

In [ ]:
# Variable + Tensor = Tensor

In [9]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())
#Conversely, to disable the default behavior of 
# watching all tf.Variables, set watch_accessed_
# variables=False when creating the gradient tape

6.0


In [10]:
# Persistent Tape
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # [4.0, 108.0] (4 * x**3 at x = [1.0, 3.0])
print(tape.gradient(y, x).numpy())  # [2.0, 6.0] (2 * x at x = [1.0, 3.0])

[  4. 108.]
[2. 6.]


In [ ]:
# Use tape context ONLY where required, it uses memory to hold intermediate results
# multiple/multivariate targets -> returns sum of gradients; multiple sources -> returns multiple outputs
# if dont need sum and need separate for multivar -> use Jacobians instead of Gradients
#

In [11]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)
dy_dx

<tf.Tensor: shape=(201,), dtype=float32, numpy=
array([4.53958055e-05, 5.01696704e-05, 5.54454418e-05, 6.12759977e-05,
       6.77195349e-05, 7.48406237e-05, 8.27104013e-05, 9.14074917e-05,
       1.01019010e-04, 1.11640838e-04, 1.23379359e-04, 1.36351780e-04,
       1.50687600e-04, 1.66530357e-04, 1.84037955e-04, 2.03385600e-04,
       2.24766321e-04, 2.48393306e-04, 2.74502789e-04, 3.03354813e-04,
       3.35237681e-04, 3.70468944e-04, 4.09399334e-04, 4.52417444e-04,
       4.99951013e-04, 5.52473066e-04, 6.10506453e-04, 6.74626906e-04,
       7.45472440e-04, 8.23745097e-04, 9.10221133e-04, 1.00575760e-03,
       1.11129810e-03, 1.22788735e-03, 1.35667447e-03, 1.49892876e-03,
       1.65605021e-03, 1.82957877e-03, 2.02121865e-03, 2.23284075e-03,
       2.46650912e-03, 2.72449665e-03, 3.00930603e-03, 3.32368701e-03,
       3.67066660e-03, 4.05357173e-03, 4.47605597e-03, 4.94213449e-03,
       5.45620080e-03, 6.02308055e-03, 6.64805667e-03, 7.33690569e-03,
       8.09594523e-03, 8.9320

In [14]:
# None Gradients common mistakes
# 1. Tensor instead of Variable
# 2. Non-TF calculations (eg. numpy)
# 3. Watched variable is non-float (int or string etc)
# 4. Variable outside tape context
# 5. Some ops do not have gradients (throw error) or are non-differentiable (return None)

In [15]:
# To return a zero isntead of None for unconnected grads
x= tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))

tf.Tensor([0. 0.], shape=(2,), dtype=float32)


### Graphs

In [ ]:
import tensorflow as tf
import timeit
from datetime import datetime

In [ ]:
# tf.function returns a tf.types.experimental.PolymorphicFunction
# object, which builds a tf graph from a python function
# Define a Python function.
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# The Python type of `a_function_that_uses_a_graph` will now be a
# `PolymorphicFunction`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a `tf.function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

In [8]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  print(x)
  x = x + b
  return x

# Using the `tf.function` decorator makes `outer_function` into a
# `PolymorphicFunction`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

In [9]:
outer_function(tf.constant([[1.0, 2.0]])).numpy()

Tensor("MatMul:0", shape=(1, 1), dtype=float32)


array([[12.]], dtype=float32)

In [10]:
# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

##### Automatic conversions inside tf.function
**`if` Statements:**
1. **Static `if`:** Python boolean, only one branch in the graph.
2. **Dynamic `if`:** Tensor condition, converted to `tf.cond` in the graph.

**`for` Loops:**
1. **Static `for`:** Iterates over a static range, unrolled in the graph.
2. **Dynamic `for`:** Tensor range, converted to `tf.while_loop`.

**`while` Loops:**
1. **Static `while`:** Python condition, resolved during tracing, no loop in the graph.
2. **Dynamic `while`:** Tensor condition, converted to `tf.while_loop` in the graph.

In [14]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# Using `tf.function` makes `tf_simple_relu` a `PolymorphicFunction` that wraps
# `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph: 1
Second branch, with graph: 0


In [15]:
# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            do_return, retval_ = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_body

In [16]:
# This is the graph itself.
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "then_branch"
    value {
      func {
        name: "cond_true_120"
      }
    }
  }
  attr {
    key: "output_shapes"
    value {
      list {
        shape {
        }
        shape {
        }
      }
    }
  }
  attr {
    key: "else_branch"
   

$\frac{-1+\sqrt{17}}{2}$

In [1]:
import math
math.sqrt(10 - 2*math.sqrt(5))

2.3511410091698925

In [2]:
math.sqrt(5) - 1

1.2360679774997898

In [4]:
3*math.sqrt(2*(5+math.sqrt(5))) + math.sqrt(10-2*math.sqrt(5))

13.763819204711734

$\boxed{\dfrac{a\,\sqrt{3}}{3}}$

\\[
\boxed{R_{2} = 2r\,\bigl(3\sqrt{3}-5\bigr) \quad \text{and} \quad \text{Area}(S_{2}) = 4\pi\,r^{2}\,\bigl(3\sqrt{3}-5\bigr)^{2}}
\\]

\\[
\frac{R_{2}}{r} = 2\,(3\sqrt{3} - 5)
\quad\text{and}\quad
\frac{\text{Area}(S_{2})}{\text{Area}(S_{1})}
= 4\,(3\sqrt{3} - 5)^{2}.
\\]

In [1]:
import sympy as sp

def fourier_cosine_transform_x2_exp_negx2over2():
  x = sp.Symbol('x', real=True, nonnegative=True)
  w = sp.Symbol('w', real=True)

  # Define the integrand f(x) = x^2 * exp(-x^2/2) * cos(w*x)
  f = x**2 * sp.exp(-x**2/sp.Integer(2)) * sp.cos(w*x)

  # Perform the integral from 0 to infinity
  integral_expr = sp.integrate(f, (x, 0, sp.oo))

  # Simplify the result
  result = sp.simplify(integral_expr)

  return result

if __name__ == "__main__":
  result_symbolic = fourier_cosine_transform_x2_exp_negx2over2()
  print("Fourier Cosine Transform = ", result_symbolic)

Fourier Cosine Transform =  sqrt(2)*sqrt(pi)*(1 - w**2)*exp(-w**2/2)/2
